Задание:
1.	Загрузить файл длиной не менее 2000 символов. 
2.	Составить программу, которая считает число уникальных слов в тексте (без критерия схожести)
3.	Составить программу, которая считает число гласных и согласных букв. 
4.	Составить программу, которая считает число предложений, их длину и число (количество) раз использования каждого слова в тексте (с критерием схожести, критерий схожести слов выбрать самостоятельно, например, spacy (en_core_web_sm) или расстояние Левенштейна). 
5.	Вывести 10 наиболее часто встречаемых слов. 

***
Все задание делится на несколько частей. Соответственно, для каждой части создаем свой jupiter notebook.
В данном блокноте выполняем пункт 2 задания.

***
Анализ по пункту 2 задания (с расширением по пункту 5 -  будет интересно сравнить, различается ли первая десятка, если уникальные слова отбираются без критерия схожести и с использованием критерия схожести):

Для подсчета уникальных слов в тексте без критерия схожести можно находить каждое следующее слово и заносить его в словарь dict в виде ключа, если этого ключа еще нет в словаре. Значение value, соответствующее этому ключу, будет присваиваться равным единице. Если же такой ключ уже есть, то соответствующее ему значение value увеличивается на единицу. После прохождения всего текста числом уникальных слов будет считаться число ключей в полученном словаре. 
Поскольку наиболее частотными будут не значащие стоп-слова типа артиклей, предлогов, местоимений, следует их предварительно удалить из текста.
Значения ключей будут далее использованы для нахождения 10 наиболее встречаемых слов. Для этого нужно отсортировать словарь по значениям полей values и взять те десять ключей, которым будут соответствует максимальное значение values
Если выяснится, что десятое по встречаемости слово встречается n раз, и имеется еще несколько слов, не включенных в десятку, но которые тоже встречаются n раз, то придется расширить десятку, включив в нее и эти слова. 

Чтобы процессу не мешали числа и знаки препинания, можно предварительно заместить их все пробелами (для этого будем с помощью встроенной функции Python проверять, относится ли каждый знак к алфавитным). 
Чтобы не было путаницы с регистрами текста, предварительно приводим весь текст к нижнему регистру.

Тогда план работы по пункту 2 следующий.

1. Загрузить текст из файла.
2. Перевести текст в нижний регистр.
3. Убрать знаки препинания и шире - все неалфавитные знаки, заменив их пробелами.
4. Разбить текст на слова, считая пробелы разделителями.
5. Из полученного списка слов убрать все стоп-слова.
6. Занести в словарь все слова из списка, для каждого указать число раз, сколько оно встречалось в списке.
7. Отсортировать полученный словарь по значению ключа (дающего частотность соответствующего слова)
8. Отобрать 10 наиболее часто встречающихся слов в полученном словаре.
9. Если в полученном словаре есть еще слова с той же частотностью, что и у десятого слова, также включить их в выводимый результат.

Загружаем текстовый файл:

In [15]:
# import

file_name = 'GospelJohn.txt' # file with text
try:
    f = open(file_name,"r") # open file for reading
    text = f.read()         # reading file 
    f.close()               # closing file
    print('Read successfully')
except:
    print('Error reading file!')

Read successfully


Переводим текст в нижний регистр.
Убираем не-алфавитные знаки и разбиваем текст на слова, помещаем их в список.

In [16]:
# to lower case:
text = text.lower()
# replace non-alphabetic signs by whitespaces:
text_no_signs = "".join([c if c.isalpha() else " " for c in text])

word_list = text_no_signs.split() # split text into words and put to the list

Убираем из списка лишние слова ("стоп-слова").
Для этого вначале создаем список стоп-слов
(источник: https://github.com/explosion/spaCy/blob/master/spacy/lang/en/stop_words.py)
Также мы немного расширяем этот список для учета устаревших слов, фигурирующих в нашем тексте

In [18]:
STOP_WORDS = set(
    """
a about above across after afterwards again against all almost alone along
already also although always am among amongst amount an and another any anyhow
anyone anything anyway anywhere are around as at

back be became because become becomes becoming been before beforehand behind
being below beside besides between beyond both bottom but by

call can cannot ca could

did do does doing done down due during

each eight either eleven else elsewhere empty enough even ever every
everyone everything everywhere except

few fifteen fifty first five for former formerly forty four from front full
further

get give go

had has have he hence her here hereafter hereby herein hereupon hers herself
him himself his how however hundred

i if in indeed into is it its itself

keep

last latter latterly least less

just

made make many may me meanwhile might mine more moreover most mostly move much
must my myself

name namely neither never nevertheless next nine no nobody none noone nor not
nothing now nowhere

of off often on once one only onto or other others otherwise our ours ourselves
out over own

part per perhaps please put

quite

rather re really regarding

same say see seem seemed seeming seems serious several she should show side
since six sixty so some somehow someone something sometime sometimes somewhere
still such

take ten than that the their them themselves then thence there thereafter
thereby therefore therein thereupon these they third this those though three
through throughout thru thus to together too top toward towards twelve twenty
two

under until up unless upon us used using

various very very via was we well were what whatever when whence whenever where
whereafter whereas whereby wherein whereupon wherever whether which while
whither who whoever whole whom whose why will with within without would

yet you your yours yourself yourselves
""".split()
)

contractions = ["n't", "'d", "'ll", "'m", "'re", "'s", "'ve"]
STOP_WORDS.update(contractions)

for apostrophe in ["‘", "’"]:
    for stopword in contractions:
        STOP_WORDS.add(stopword.replace("'", apostrophe))

obsoletisms = ["art", "hath", "thou", "thee", "unto"] # obsolete word forms
STOP_WORDS.update(obsoletisms)

Убираем из списка слова, если они входят в список стоп-слов:

In [20]:
word_list_new = list()
for aword in word_list:
    if aword not in STOP_WORDS:
        word_list_new.append(aword)

# print(len(word_list))
# print(len(word_list_new))
word_list = word_list_new


Проходим по всему набору слов и вносим ранее не встреченные слова в словарь, для ранее встреченных увеличиваем value на единицу

In [21]:
word_dict = dict() # empty dictionary for all words
# putting all the words into dict:
for aword in word_list:
    if aword in word_dict: # if already in dict
        word_dict[aword] += 1            
    else:                  # if not in dict yet
        word_dict[aword] = 1  
                 
# print(word_dict)

Находим число уникальных слов (без критерия похожести; "стоп-слова" уже удалены и не учитываются)

In [22]:
key_number = len(word_dict.keys())
print(f'The number of the unique words in the text: {key_number}')

The number of the unique words in the text: 170


Сортируем словарь по увеличению частоты встречаемости слова, извлекаем десять последних (то есть самых частых) слов.
Если 11-е, 12-е и т.д. слова встречаются так же часто, как и десятое, выводим также и их, потому что по факту они все делят десятое место в списке наиболее частых слов

In [23]:
# sorting the list of the most frequent words:
word_dict_sorted = sorted(word_dict.items(), key = lambda item: item[1])
print('The most frequent words:')

# extracting the most frequent words
for i in range(10):
    aword = word_dict_sorted[-i-1][0]
    frequency = word_dict_sorted[-i-1][1]
    print(f'{i+1}. The word "{aword}" has the frequency {frequency}')

# extracting also the 11th, 12th etc, if the frequency is the same as for the 10th one:
while word_dict_sorted[-i-2][1] == word_dict_sorted[-10][1]:
    aword = word_dict_sorted[-i-2][0]
    frequency = word_dict_sorted[-i-2][1]
    print(f'{i}. The word "{aword}" has frequency {frequency}')
    i += 1

The most frequent words:
1. The word "saith" has the frequency 13
2. The word "said" has the frequency 12
3. The word "jesus" has the frequency 12
4. The word "god" has the frequency 12
5. The word "john" has the frequency 9
6. The word "saw" has the frequency 7
7. The word "son" has the frequency 6
8. The word "sent" has the frequency 6
9. The word "man" has the frequency 6
10. The word "light" has the frequency 6


Итак, нами был проделан анализ англоязычного текста в отношении количества уникальных слов без объединения словоформ по критерию схожести.

Выводы:
1. При анализе числа уникальных слов в тексте следует удалять т. наз. "стоп-слова", то есть артикли, предлоги и прочие слова, не несущие смысловой нагрузки.
2. Для исключения ситуации, когда слова в начале и середине предложения считаются за разные слова за счет отличия регистра первой буквы, следует привести слова перед анализом к одному регистру.
3. В классическом англоязычном тексте 17 в. на ~1000 слов (4102 знака без учета пробелов) после удаления стоп-слов оказалось 170 уникальных слов (без дальнейшего объединения словоформ по критерию схожести), то есть число уникальных значащих слов примерно в пять раз меньше, чем общее число всех слов.
4. Десять отобранных уникальных слов с максимальной частотностью дают хорошее представление о тематике текста.